In [1]:
import os 
os.environ["SPARK_HOME"] = "/home/com3dian/Documents/github/Period4/DIS/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession

## You can add more config while building 
spark = SparkSession.builder.master("local[8]").\
                    config("spark.app.name","session_one").\
                    getOrCreate() #number of threads = 16

In [2]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
import numpy as np


In [3]:
def dataframe2NumpyArray(df, colName):
    '''
    convert spark dataframe to numpy array
    '''
    return np.array(df.select(colName).collect())


def numpyArray2Matrix(array):
    '''
    convert numpy array to spark Rowmatrix
    ----------------------------
    return: Rowmatrix
    '''
    if len(array.shape) == 3:
        array.reshape((array.shape[0], array.shape[-1]))
    
    
    denseVectorList = []
    for i in range(len(array)):
        denseVectorList.append(Vectors.dense(array[i]))
        
    
    RDD = spark.sparkContext.parallelize(denseVectorList)
    normVectors = RDD.map(lambda x: x/(np.linalg.norm(x, 2)))
    
    RDD = spark.sparkContext.parallelize(normVectors.collect())
    matrix = RowMatrix(RDD)
        
    return matrix


def SVDsimilarity(matrix, numDimension = 1, normalization = False):
    '''
    generalized cosine similarity using SVD(singular value decomposition)
    by doing SVD, the input matrix Y will be decomposited into 3 matrix: U, S, V, with Y = USV^T
    where S can be considered as a lower rank approximation of Y
    the SVD optimal in the sense that minimizing the Frobinius norm of reconstruction error || \hat{Y} - Y ||^{2}_{F}
    therefore, by comparing the 'order K coefficient of determination' \frac{||\hat{Y} ||^2_F}{||Y ||^2_F}, we shall a similarity.
    ----------------------
    in the case of only 2 vectors, the SVD similarity is equal to the cosine similarity
    ----------------------
    the original SVD similarity is ranged from 1/n to 1, where n is the number of vectors
    to get it can range over the entire [0,1] interval, one can normalize it by \frac{}{} if only using the first sigular value
    
    ------------------------------------------
    matrix: pyspark RowMatrix, represents a row-oriented distributed Matrix with no meaningful row indices
            each column/row is an input vector
            all element in matrix should be positive
    numDimension: integer, if not 1 then use the first(largest) few singular value
    normalization: if true then do normalization
    
    '''
    N = matrix.numRows()
    # SVD
    svd = matrix.computeSVD(numDimension, computeU=False)
    sVector = svd.s.toArray()
    YApproximate = np.sum(sVector*sVector)**0.5
    
    GramianMatrix = matrix.computeGramianMatrix().toArray()
    Y = np.trace(GramianMatrix)**0.5
    
    # normalization
    if not normalization:
        similarityScore = YApproximate/Y
    else:
        similarityScore = ((YApproximate/Y * N) - 1)/(N -1)
        
    # return 2 * similarityScore**2 - 1
    # double angle formula
    return 2 * similarityScore**2 - 1

In [4]:
def crossHomogeneityScore(df, queryColName, featureColName):
    '''
    
    '''
    if queryColName not in df.schema.names:
        
        npArray = dataframe2NumpyArray(df, featureColName)
        matrix = numpyArray2Matrix(npArray)
        similarity = SVDsimilarity(matrix)
        
        return similarity
    
    
    totalRows = df.count()
    queries = list(set(df.select(queryColName).collect()))
    
    homogeneityScore = 0
    
    for query in queries:
        # get each cluster
        dfQuery = df.filter(df[queryColName] == query.query)
        # get number of rows
        numRows = dfQuery.count()
        
        npArray = dataframe2NumpyArray(dfQuery, featureColName)
        matrix = numpyArray2Matrix(npArray)
        
        similarity = SVDsimilarity(matrix)
        homogeneityScore += similarity * numRows/totalRows
    
    return homogeneityScore

In [5]:
df = spark.read.csv("/home/com3dian/Documents/github/DIS_project_2022/data/testpeople.csv",header=True,inferSchema=True)
df.show()

+----+---------+---+
|Name|     city|age|
+----+---------+---+
|John|  utrecht| 20|
|Mary|amsterdam| 21|
|Nick|rotterdam| 22|
|Nick|  utrecht| 21|
|John|rotterdam| 21|
|Mary|rotterdam| 22|
|Nick|  utrecht| 20|
|John|rotterdam| 21|
|Nick|amsterdam| 20|
|Mary|  utrecht| 22|
+----+---------+---+



In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

def listOfFrequencyTables(df): #take main dataframe, generate frequency dataframes
    '''
    
    '''
    histograms = []
    for col in df.dtypes:
        h=df.groupBy(col[0]).count()
        h = h.sort(desc("count"))
        histograms.append(h)
        h.show() #comment this line to suppress output
    return histograms

histograms = listOfFrequencyTables(df)

+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+

+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    4|
|rotterdam|    4|
|amsterdam|    2|
+---------+-----+

+---+-----+
|age|count|
+---+-----+
| 21|    4|
| 22|    3|
| 20|    3|
+---+-----+



In [7]:
def getDecompFromTopFrequencies(df, histograms):
    clusterlst=[]
    for i in range(len(histograms)): #query database with top values of all columns
        d= str(histograms[i].first()) #value of the first row
        #print(d)
        d = d.split(",")[0].split('=')[1] #the splits are for formatting the string
        #print(d)
        #print("d before:",d)
        if "'"  in d:
            d = d.split("'")[1]
        
        #print("d after:",d)
        #print(type(d))
        cname = str(histograms[i][0]).split("'")[1]
        print(cname,"=",d)
        
        data = (df.filter(col(cname) == d))
        
        data.show(15)
        clusterlst.append(data)
    return clusterlst

clusterlst = getDecompFromTopFrequencies(df, histograms)


Name = Nick
+----+---------+---+
|Name|     city|age|
+----+---------+---+
|Nick|rotterdam| 22|
|Nick|  utrecht| 21|
|Nick|  utrecht| 20|
|Nick|amsterdam| 20|
+----+---------+---+

city = rotterdam
+----+---------+---+
|Name|     city|age|
+----+---------+---+
|Nick|rotterdam| 22|
|John|rotterdam| 21|
|Mary|rotterdam| 22|
|John|rotterdam| 21|
+----+---------+---+

age = 21
+----+---------+---+
|Name|     city|age|
+----+---------+---+
|Mary|amsterdam| 21|
|Nick|  utrecht| 21|
|John|rotterdam| 21|
|John|rotterdam| 21|
+----+---------+---+



In [8]:
def are_dfs_equal(df1, df2): #this works, i tested it
    res = df1.subtract(df2) #set subtraction on the two dataframes. 
    if res.count() == 0: #subtraction yielded empty set
        print("dataframes are equal")
        return True
    else:
        print("error! these rows are not in the union of your queries:")
        res.show() #show which tuples are not included in your query union
        return False

from functools import reduce
from pyspark.sql.functions import lit
from pyspark.sql import DataFrame

def getDecompUsingFreqTable(df,freqdf): #takes original database and one frequency table as input, returns union of all queried dataframes as output
    print("this is frequency table:")
    freqdf.show()
    cname = freqdf.columns[0]
    valuelist = (freqdf.select(freqdf.columns[0]).rdd.flatMap(lambda x: x).collect()) #list of all values of frequency column
    
    unionlst = []
    
    for v in valuelist: #each unique value in the freq. table is used as a query
        result = df.filter(col(cname) == v)
        querystr = cname + "=" + str(v)
        print("querystr:",querystr)
        
        containsquery = False
        for c in df.columns: #check if query column exists in the input dataframe
            if "query" in c:
                containsquery= True
        
        if(containsquery): #check if query column already exists in the input
            result=result.withColumn("query1",lit(querystr))
            print("first newres:")
            
            result= result.withColumn("joined",concat(concat(col("query"), lit(","), col("query1")))) #putting query with existing queries
            print("after join")
            
            columns_to_drop = ['query', 'query1']
            
            result=result.drop('query')
            result=result.drop('query1')
            result=result.withColumnRenamed("joined","query")
            
            result=result
        else:
            print("creating query column:")
            result=result.withColumn("query",lit(querystr))
        print(querystr)
        
        unionlst.append(result)
    unn = reduce(DataFrame.unionAll, unionlst) #put all queried dataframes back together as one
    
    
    return unn


union = (getDecompUsingFreqTable(df,histograms[0])) #function call with 'Name' frequency table
print("equality result:")
#subdf = are_dfs_equal(df,union.drop('query')) #checks if union of queries covers whole database
#print(subdf)
print("\nnext run:")
union1 = (getDecompUsingFreqTable(union,histograms[1]))

union2 = (getDecompUsingFreqTable(union1,histograms[2]))

print("after queries:\n")
union.show(10,False)
union1.show(10,False)
union2.show(10,False)

this is frequency table:
+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+

querystr: Name=Nick
creating query column:
Name=Nick
querystr: Name=Mary
creating query column:
Name=Mary
querystr: Name=John
creating query column:
Name=John
equality result:

next run:
this is frequency table:
+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    4|
|rotterdam|    4|
|amsterdam|    2|
+---------+-----+

querystr: city=rotterdam
first newres:
after join
city=rotterdam
querystr: city=utrecht
first newres:
after join
city=utrecht
querystr: city=amsterdam
first newres:
after join
city=amsterdam
this is frequency table:
+---+-----+
|age|count|
+---+-----+
| 21|    4|
| 22|    3|
| 20|    3|
+---+-----+

querystr: age=21
first newres:
after join
age=21
querystr: age=22
first newres:
after join
age=22
querystr: age=20
first newres:
after join
age=20
after queries:

+----+---------+---+---------+
|Name|city     |age|query    |
+----+---------+--

In [9]:
histograms[0].show()

+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+



In [10]:
def addFeatureVector(df): #get feature vector for any dataframe for homogeneity function
    string_cols = [c for c, t in df.dtypes if t =='string' and c != 'query'] #get all columns that have stringtype, except query column
    
    stringindex_cols = [(i + "_indexed") for i in string_cols]
    indexer  = StringIndexer( inputCols=string_cols, outputCols=stringindex_cols, handleInvalid='error', stringOrderType='frequencyDesc')
    indexer.setHandleInvalid("keep") #change to "skip" to remove problematic rows
    indexed = indexer.fit(df).transform(df) #dataframe with indexed columns attached
    
    allnonstringcols = [column.name for column in indexed.schema if column.dataType != StringType()]
    vecAssembler = VectorAssembler(outputCol="features")
    
    # normalizaing
    for col in allnonstringcols:
        maxValue = indexed.select(max(col)).collect()[0][0]
        minValue = indexed.select(min(col)).collect()[0][0]
        indexed = indexed.withColumn(col + '_normalized', (indexed[col] - minValue)/(maxValue - minValue))
    
    allnonstringcols = [col + '_normalized' for col in allnonstringcols]
    vecAssembler.setInputCols(allnonstringcols) #all numerical columns are put into feature vector, including indexed cols
    result=  ( vecAssembler.transform(indexed)) #return the dataframe with feature column attached
    
    for col in allnonstringcols:
        result = result.drop(col)
    for col in stringindex_cols:
        result = result.drop(col)
    return result

union2withvec = addFeatureVector(union2)
union2withvec.show()

+----+---------+---+--------------------+-------------+
|Name|     city|age|               query|     features|
+----+---------+---+--------------------+-------------+
|John|rotterdam| 21|Name=John,city=ro...|[0.5,0.5,0.0]|
|John|rotterdam| 21|Name=John,city=ro...|[0.5,0.5,0.0]|
|Nick|  utrecht| 21|Name=Nick,city=ut...|[0.5,0.0,0.5]|
|Mary|amsterdam| 21|Name=Mary,city=am...|[0.5,1.0,1.0]|
|Nick|rotterdam| 22|Name=Nick,city=ro...|[1.0,0.0,0.0]|
|Mary|rotterdam| 22|Name=Mary,city=ro...|[1.0,1.0,0.0]|
|Mary|  utrecht| 22|Name=Mary,city=ut...|[1.0,1.0,0.5]|
|Nick|  utrecht| 20|Name=Nick,city=ut...|[0.0,0.0,0.5]|
|John|  utrecht| 20|Name=John,city=ut...|[0.0,0.5,0.5]|
|Nick|amsterdam| 20|Name=Nick,city=am...|[0.0,0.0,1.0]|
+----+---------+---+--------------------+-------------+



In [11]:
crossHomogeneityScore(union2withvec, 'query', 'features')

1.0

In [21]:
def shuffle(hist):
    return sorted(hist, key = lambda x: x.count())


def getDecompositionbyColumn(df, K):
    '''
    compute the final decomposition
    
    '''
    histograms = shuffle(listOfFrequencyTables(df))
    decomUnion = df
    colLeft = len(histograms)
    decomUnionWithVec = addFeatureVector(decomUnion)
    overAllHomoScore = crossHomogeneityScore(decomUnionWithVec, 'query', 'features')
    nBuckets = 1
    
    ifUpdated = True
    while nBuckets <= K and colLeft > 0 and ifUpdated:
        # at least one column left
        # fewer groups than K

        ifUpdated = False
        
        for freqdf in histograms:
            
            unionWithVec = getDecompUsingFreqTable(decomUnionWithVec, freqdf)
            # unionWithVec.show()
            crossScore =  crossHomogeneityScore(unionWithVec, 'query', 'features')
            
            nBuckets = len(unionWithVec.select('query').distinct().collect())
            
            if crossScore > overAllHomoScore and nBuckets <= K :
                # 
                overAllHomoScore = crossScore
                coldf = freqdf
                ifUpdated = True
                
        if ifUpdated:
            decomUnionWithVec = getDecompUsingFreqTable(decomUnionWithVec, freqdf)
            histograms.remove(coldf)
        colLeft = len(histograms)
        
    nBuckets = len(decomUnionWithVec.select('query').distinct().collect())
    print('user requested K =', str(K), ', but we can only got ', str(nBuckets), 'clusters.') if nBuckets != K
    return decomUnionWithVec.drop('features')
            
            
        
            
        
    

In [22]:
result = getDecompositionbyColumn(df, 3)

+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+

+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    4|
|rotterdam|    4|
|amsterdam|    2|
+---------+-----+

+---+-----+
|age|count|
+---+-----+
| 21|    4|
| 22|    3|
| 20|    3|
+---+-----+

this is frequency table:
+----+-----+
|Name|count|
+----+-----+
|Nick|    4|
|Mary|    3|
|John|    3|
+----+-----+

querystr: Name=Nick
creating query column:
Name=Nick
querystr: Name=Mary
creating query column:
Name=Mary
querystr: Name=John
creating query column:
Name=John
this is frequency table:
+---------+-----+
|     city|count|
+---------+-----+
|  utrecht|    4|
|rotterdam|    4|
|amsterdam|    2|
+---------+-----+

querystr: city=rotterdam
creating query column:
city=rotterdam
querystr: city=utrecht
creating query column:
city=utrecht
querystr: city=amsterdam
creating query column:
city=amsterdam
this is frequency table:
+---+-----+
|age|count|
+---+-----+
| 21|    4|
| 22|    3

In [20]:
result.show(result.count(), False)

+----+---------+---+------+
|Name|city     |age|query |
+----+---------+---+------+
|Mary|amsterdam|21 |age=21|
|Nick|utrecht  |21 |age=21|
|John|rotterdam|21 |age=21|
|John|rotterdam|21 |age=21|
|Nick|rotterdam|22 |age=22|
|Mary|rotterdam|22 |age=22|
|Mary|utrecht  |22 |age=22|
|John|utrecht  |20 |age=20|
|Nick|utrecht  |20 |age=20|
|Nick|amsterdam|20 |age=20|
+----+---------+---+------+

